In [87]:
"""
File : data_for_lstm.ipynb
Created by : Amen Ouannes
Brief : preparing the full dataset and adapting it to LSTM further modifications

"""

'\nFile : data_for_lstm.ipynb\nCreated by : Amen Ouannes\nBrief : preparing the full dataset and adapting it to LSTM further modifications\n\n'

In [57]:
#import libraries
import data_O as clean
import RV_tools as tools
import AX3_tools as ax3
import importlib
import os as os
import pandas as pd
import pickle as pk
import numpy as np
import pyarrow.feather as feather
from datetime import datetime
importlib.reload(tools)
importlib.reload(ax3)
print(os.getcwd())

/data/amen/amen_env/data_processing


### Ruuvi data treatement

In [58]:
filenames = ["/data/amen/amen_env/data_processing/data/0405_1.csv","/data/amen/amen_env/data_processing/data/0405_2.csv",
             "/data/amen/amen_env/data_processing/data/0405_3.csv","/data/amen/amen_env/data_processing/data/0405_4.csv",
             "/data/amen/amen_env/data_processing/data/0505_1.csv","/data/amen/amen_env/data_processing/data/0505_2.csv",
             "/data/amen/amen_env/data_processing/data/0505_3.csv","/data/amen/amen_env/data_processing/data/0505_4.csv",
             "/data/amen/amen_env/data_processing/data/0605_1.csv","/data/amen/amen_env/data_processing/data/0605_2.csv",
             "/data/amen/amen_env/data_processing/data/0605_3.csv","/data/amen/amen_env/data_processing/data/0605_4.csv",
             "/data/amen/amen_env/data_processing/data/0705_1.csv","/data/amen/amen_env/data_processing/data/0705_2.csv",
             "/data/amen/amen_env/data_processing/data/0705_3.csv","/data/amen/amen_env/data_processing/data/0705_4.csv",
             "/data/amen/amen_env/data_processing/data/0805_1.csv","/data/amen/amen_env/data_processing/data/0805_2.csv",
             "/data/amen/amen_env/data_processing/data/0805_3.csv","/data/amen/amen_env/data_processing/data/0805_4.csv",
             "/data/amen/amen_env/data_processing/data/0905_1.csv","/data/amen/amen_env/data_processing/data/0905_2.csv",
             "/data/amen/amen_env/data_processing/data/0905_3.csv","/data/amen/amen_env/data_processing/data/0905_4.csv",
             "/data/amen/amen_env/data_processing/data/1005_1.csv","/data/amen/amen_env/data_processing/data/1005_2.csv",
             "/data/amen/amen_env/data_processing/data/1005_3.csv","/data/amen/amen_env/data_processing/data/1005_4.csv",
             "/data/amen/amen_env/data_processing/data/1105_1.csv"]
macs= {"D1:B6:FC:34:99:3A" : 7480, "F4:CC:E1:C5:8A:A9" :8855,
       "FB:E5:24:5B:8A:68" : 8851, "D0:C9:50:5E:88:F8" :8846, "D3:6B:03:69:94:4E" : 8854}

In [59]:
#state the list of calves
calf_4 = "D1:B6:FC:34:99:3A"
calf_11 = "D0:C9:50:5E:88:F8"
calf_7 = "F4:CC:E1:C5:8A:A9"
calf_3 = "FB:E5:24:5B:8A:68"
calf_1 = "D3:6B:03:69:94:4E"
#create a dictionnary to contain all of my calves
calves = {calf_1 : pd.DataFrame(), calf_4 : pd.DataFrame(), calf_11 : pd.DataFrame()
          , calf_7 : pd.DataFrame(), calf_3 : pd.DataFrame()}

In [60]:
### Prepare the ruuvi data for concatenation
for filename in filenames:
    #create a clean dataframe
    data = clean.processing(filename)
    #classify new data by calf
    calves = tools.classify(data, calves)

print("ruuvi data ready!")

ruuvi data ready!


In [61]:
for key, value in calves.items():
    # filter raw data and generate features
    calves[key] = tools.filter(value)
    
    #change macs to id
    calves[key].rename(columns={'mac': 'calfNumber'}, inplace = True)
    calves[key]=calves[key].replace(macs)
    
calves[calf_1].head()

,time,acc_x,acc_y,acc_z,st_dev_x,st_dev_y,st_dev_z,ax_dynamic,ay_dynamic,az_dynamic,calfNumber
0,2024-05-04 00:00:00,-0.4440,-0.040000,-0.932000,0.015319,0.009238,0.008000,0.344703,-0.001977,-1.015600,8854
1,2024-05-04 00:00:01,-0.4368,-0.036800,-0.924800,0.007729,0.007729,0.014703,0.351903,0.001223,-1.008400,8854
2,2024-05-04 00:00:02,-0.4460,-0.030000,-0.926000,0.005657,0.010254,0.013352,0.342703,0.008023,-1.009600,8854
3,2024-05-04 00:00:03,-0.4320,-0.036923,-0.916923,0.006532,0.007686,0.010087,0.356703,0.001100,-1.000523,8854
4,2024-05-04 00:00:04,-0.4400,-0.035200,-0.929600,0.011314,0.010119,0.011806,0.348703,0.002823,-1.013200,8854


In [62]:
#Ensure the existence of all accelerations
for key, value in calves.items():
    nan_rows = value[value.isna().any(axis=1)]
    assert(len(nan_rows)==0)
print('assertion is correct!')
for key, value in calves.items():
    print(f'length of the calf {key} : {len(value)}')

assertion is correct!
length of the calf D3:6B:03:69:94:4E : 577736
length of the calf D1:B6:FC:34:99:3A : 585956
length of the calf D0:C9:50:5E:88:F8 : 583621
length of the calf F4:CC:E1:C5:8A:A9 : 584282
length of the calf FB:E5:24:5B:8A:68 : 584209


In [63]:
assert(len(calves[calf_1]) < 7*24*3600) #number of seconds collected is less than the optimal number of seconds
print("Missing data due to the inactivity of the ruuvi sensors in multiple times")
for key, value in calves.items():
    print(f'missing data for calf {key} : {7*24*3600 + 1 - len(value)}')

Missing data due to the inactivity of the ruuvi sensors in multiple times
missing data for calf D3:6B:03:69:94:4E : 27065
missing data for calf D1:B6:FC:34:99:3A : 18845
missing data for calf D0:C9:50:5E:88:F8 : 21180
missing data for calf F4:CC:E1:C5:8A:A9 : 20519
missing data for calf FB:E5:24:5B:8A:68 : 20592


### AX3 data version 1

In [67]:
#extract data from AX3 files 
AX3 = {}
AX3[calf_4] = pd.read_csv("/data/amen/amen_env/data_processing/data/79404_0000000000.cwa.up.down.hmm.sec.tab", sep='\t')
AX3[calf_7] = pd.read_csv("/data/amen/amen_env/data_processing/data/78620_0000000000.cwa.up.down.hmm.sec.tab", sep='\t')
AX3[calf_3] = pd.read_csv("/data/amen/amen_env/data_processing/data/80625_0000000000.cwa.up.down.hmm.sec.tab", sep='\t')
AX3[calf_11] = pd.read_csv("/data/amen/amen_env/data_processing/data/82291_0000000000.cwa.up.down.hmm.sec.tab", sep='\t')
AX3[calf_1] = pd.read_csv("/data/amen/amen_env/data_processing/data/79959_0000000000.cwa.up.down.hmm.sec.tab", sep='\t')


In [68]:
list_id = [7480, 8855, 8851, 8846, 8854]
i = 0
for key, value in AX3.items():
    AX3[key] = ax3.apply(value, list_id[i])
    i += 1

AX3[calf_1].head()

,time,state,calfNumber
719520,2024-05-04 00:00:00,down,8854
719521,2024-05-04 00:00:01,down,8854
719522,2024-05-04 00:00:02,down,8854
719523,2024-05-04 00:00:03,down,8854
719524,2024-05-04 00:00:04,down,8854


### AX3 data version 2

In [66]:
#extract data from AX3 files 
AX3 = {}
AX3[calf_4] = feather.read_table("/data/amen/amen_env/data_processing/data/79404_0000000000.cwa.sec.hmm.sec.feather")
AX3[calf_7] = feather.read_table("/data/amen/amen_env/data_processing/data/78620_0000000000.cwa.sec.hmm.sec.feather")
AX3[calf_3] = feather.read_table("/data/amen/amen_env/data_processing/data/80625_0000000000.cwa.sec.hmm.sec.feather")
AX3[calf_11] = feather.read_table("/data/amen/amen_env/data_processing/data/82291_0000000000.cwa.sec.hmm.sec.feather")
AX3[calf_1] = feather.read_table("/data/amen/amen_env/data_processing/data/79959_0000000000.cwa.sec.hmm.sec.feather")

In [ ]:
list_id = [7480, 8855, 8851, 8846, 8854]
i = 0
for key, value in AX3.items():
    temp = ax3.treat(value)
    temp['calfNumber'] = list_id[i]
    temp = clean.trunc(temp, "2024-05-04 00:00:00.0", "2024-05-11 00:00:00.0")
    i += 1
    AX3[key] = temp
print(AX3[calf_1]['state'].value_counts())

In [69]:
for key, value in AX3.items():
    nan_rows = value[value.isna().any(axis=1)] #be sure there's no missing values
    assert(len(nan_rows)==0)
print('assertion is correct!')
for key, value in AX3.items():
    print(f'length of the calf {key} : {len(value)}')
print('no missing data!')    

assertion is correct!
length of the calf D1:B6:FC:34:99:3A : 604801
length of the calf F4:CC:E1:C5:8A:A9 : 604801
length of the calf FB:E5:24:5B:8A:68 : 604801
length of the calf D0:C9:50:5E:88:F8 : 604801
length of the calf D3:6B:03:69:94:4E : 604801
no missing data!


### DAL data treatement

In [70]:
with open('DAL.pkl', 'rb') as f:
    DAL = pk.load(f)

DAL['state'] = 'drink'
print(DAL)

                      time calfNumber  state
0      2024-05-04 04:13:15       8851  drink
1      2024-05-04 04:13:16       8851  drink
2      2024-05-04 04:13:17       8851  drink
3      2024-05-04 04:13:18       8851  drink
4      2024-05-04 04:13:19       8851  drink
...                    ...        ...    ...
193248 2024-05-10 21:03:34       8855  drink
193249 2024-05-10 21:03:35       8855  drink
193250 2024-05-10 21:03:36       8855  drink
193251 2024-05-10 21:03:37       8855  drink
193252 2024-05-10 21:03:38       8855  drink

[193253 rows x 3 columns]


In [86]:
for key, value in AX3.items():    
    calf = value[value['state']=='down']
    drink_1 = DAL[DAL['calfNumber'] == macs[key]]
    confusion = calf[calf[['time']].isin(drink_1[['time']].to_dict(orient='list')).all(axis=1)]
    print(f'the degree of confusion is :{len(confusion)/len(calf)*100:.3f}%')

the degree of confusion is :15.280%
the degree of confusion is :0.010%
the degree of confusion is :5.796%
the degree of confusion is :0.011%
the degree of confusion is :0.013%


### Merge

In [25]:
#Merge ruuvi and DAL
def merge_(ruuvi, AX3, DAL):
    #merge ruuvi and dal
    copy_ruuvi = ruuvi.copy()
    drink = pd.merge(copy_ruuvi, DAL, on=['time', 'calfNumber'])
    drink['state'] = 'drink'
    
    #merge ruuvi and AX3
    copy_ruuvi = ruuvi.copy()
    up_down = pd.merge(copy_ruuvi, AX3, on = ['time', 'calfNumber'])
    #up_down['state'].replace({1: 'up', 0: 'down'}, inplace=True)
    
    #eliminate rows that verify drink within stand
    drink_copy = drink.copy()
    #up = up_down[up_down['state'] == 'up']
    common_rows = pd.merge(drink_copy, up_down, on=['time', 'calfNumber'], how='inner')
    #print(common_rows[common_rows['state_y'] == 'down'])
    up_down = up_down[~up_down[['time', 'calfNumber']].isin(common_rows[['time', 'calfNumber']].to_dict(orient='list')).all(axis=1)]
    
    #filter the down rows together 
    #down = up_down[up_down['state'] == 'down']
    #concatenate all
    dataset = pd.concat([up_down, drink], ignore_index=True)
    #print(dataset[dataset['state'] == 'drink'])
    dataset = dataset.sort_values(by='time')
    return dataset


**NEW MERGE**

In [71]:
def new_merge(ruuvi, ax3, drink):
    #merge and keep the NaN values
    data = pd.merge(ax3, ruuvi, on=['time', 'calfNumber'], how='outer')
    #identify the rows that should have the value drink
    mask = data['time'].isin(drink['time'])
    
    # Use the mask to set the 'state' to 'drink' for matching rows
    data.loc[mask, 'state'] = 'drink'
    assert(len(data[data.isna().any(axis=1)]) == 1+7*24*3600 -len(ruuvi)) #identify the number of missing data is also included
    new_order = ['time', 'acc_x', 'acc_y', 'acc_z','st_dev_x', 'st_dev_y', 'st_dev_z', 'ax_dynamic', 'ay_dynamic', 'az_dynamic', 'calfNumber', 'state']
    data_reordered = data[new_order]
    return data_reordered

In [72]:
i = 0
dataset = {}
for key, value in calves.items():
    i += 1
    ax3 = AX3[key]
    drink = DAL[DAL['calfNumber'] == macs[key]]
    #merge for each calf
    calf = new_merge(value, ax3, drink)
    dataset[key] = calf 
print('merge is complete!')

merge is complete!


In [81]:
dataset[calf_1].head()

,time,acc_x,acc_y,acc_z,st_dev_x,st_dev_y,st_dev_z,ax_dynamic,ay_dynamic,az_dynamic,calfNumber,state
0,2024-05-04 00:00:00,-0.4440,-0.040000,-0.932000,0.015319,0.009238,0.008000,0.344703,-0.001977,-1.015600,8854,down
1,2024-05-04 00:00:01,-0.4368,-0.036800,-0.924800,0.007729,0.007729,0.014703,0.351903,0.001223,-1.008400,8854,down
2,2024-05-04 00:00:02,-0.4460,-0.030000,-0.926000,0.005657,0.010254,0.013352,0.342703,0.008023,-1.009600,8854,down
3,2024-05-04 00:00:03,-0.4320,-0.036923,-0.916923,0.006532,0.007686,0.010087,0.356703,0.001100,-1.000523,8854,down
4,2024-05-04 00:00:04,-0.4400,-0.035200,-0.929600,0.011314,0.010119,0.011806,0.348703,0.002823,-1.013200,8854,down


**Multiple non consistent and big interval prevent from applying a linear interpolation 
So we use a filler function that fills NaN values with generic values related to the states**

In [77]:
def filler(df):
    # Define the columns to be filled and the time column
    numeric_cols = ['acc_x', 'acc_y', 'acc_z','st_dev_x', 'st_dev_y', 'st_dev_z', 'ax_dynamic', 'ay_dynamic', 'az_dynamic']
    time_col = 'time'  # Replace 'time' with the actual name of your time column
    
    # Convert only numeric columns to numeric, leaving non-numeric and time columns intact
    df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

    # Separate data by state
    drink = df[df['state'] == 'drink']
    up = df[df['state'] == 'up']
    down = df[df['state'] == 'down']
    
    # Calculate mean for each state
    drink_row = drink[numeric_cols].mean()
    up_row = up[numeric_cols].mean()
    down_row = down[numeric_cols].mean()
    
    # Fill NaN values with the mean of respective states
    df.loc[df['state'] == 'drink', numeric_cols] = df[df['state'] == 'drink'][numeric_cols].fillna(drink_row)
    df.loc[df['state'] == 'up', numeric_cols] = df[df['state'] == 'up'][numeric_cols].fillna(up_row)
    df.loc[df['state'] == 'down', numeric_cols] = df[df['state'] == 'down'][numeric_cols].fillna(down_row)
    
    # Verify there's no more empty rows
    assert(len(df[df.isna().any(axis=1)]) == 0)
    
    return df

In [78]:
for key, value in dataset.items():
    dataset[key] = filler(value)

0
0
0
0
0


In [85]:
for key, value in dataset.items():
    print(f"{key}: {value.shape}")

D3:6B:03:69:94:4E: (604801, 12)
D1:B6:FC:34:99:3A: (604801, 12)
D0:C9:50:5E:88:F8: (604801, 12)
F4:CC:E1:C5:8A:A9: (604801, 12)
FB:E5:24:5B:8A:68: (604801, 12)


### Save the respective dataset

In [26]:
#full dataset
with open('full_dataset.pkl','wb') as f:
    pk.dump(dataset,f)

In [73]:
#One minute regroupement dataset
with open('dataset_min.pkl', 'wb') as f :
    pk.dump(dataset, f)

In [54]:
#5 min regroupement dataset
with open('dataset_5min.pkl', 'wb') as f :
    pk.dump(dataset, f)

In [80]:
#AX3 version1 dataset
with open('full_datasetv1.pkl', 'wb') as f:
    pk.dump(dataset, f)